In [1]:
import os
import pandas as pd
import numpy as np
import zarr
from pathlib import Path
from seqdata import Table, FlatFASTA, GenomeFASTA, BigWig, BAM
import seqdata as sd
import xarray as xr
import pysam
import random
import pickle
import pyBigWig

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
# Make a temporary directory for the output
os.makedirs(Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tmp', exist_ok=True)

In [3]:
# Get infiles
variable_fasta_in = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'variable.fa'
variable_bed_in = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'variable.bed'
fixed_bed_in = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'fixed.bed'
out = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tmp'

# `GenomeFasta`

In [4]:
# Read in data with Python as true representation
def read_fasta(file_path):
    sequences = {}
    with open(file_path, 'r') as file:
        sequence_id = None
        sequence_lines = []
        for line in file:
            line = line.strip()
            if line.startswith(">"):
                if sequence_id:
                    sequences[sequence_id] = ''.join(sequence_lines)
                sequence_id = line[1:]  # Remove the '>'
                sequence_lines = []
            else:
                sequence_lines.append(line)
        if sequence_id:
            sequences[sequence_id] = ''.join(sequence_lines)
    return sequences


def test_GenomeFASTA(input_fasta, input_bed, temp_dir, batch_size=50, fixed_length=20, length_dim="_length"):
    """
    Tests the GenomeFASTA class for a single input FASTA file and BED coordinates.

    Parameters:
        input_fasta (str or Path): Path to the input FASTA file.
        bed (list): List of tuples (chromosome, start, end) representing BED coordinates.
        temp_dir (str or Path): Directory where temporary files will be stored.
        batch_size (int): Batch size for the GenomeFASTA reader.
        fixed_length (int): Fixed length for the sequences when writing to Zarr.
        length_dim (str): Name of the length dimension in the Zarr output.

    Raises:
        AssertionError: If any test fails.
    """
    temp_dir = Path(temp_dir)
    temp_dir.mkdir(parents=True, exist_ok=True)
    fasta_out = temp_dir / 'output.fasta.zarr'

    bed = pd.read_csv(input_bed, sep="\t", header=None)
    fasta_sequences = read_fasta(input_fasta)
    if fixed_length:
        midpoint = (bed[1] + bed[2]) // 2
        bed[1] = midpoint - fixed_length // 2
        bed[2] = midpoint + fixed_length // 2
    true = [fasta_sequences[chrom][start:end] for chrom, start, end in bed.values]
    bed["strand"] = "+"

    # Test instantiation of the GenomeFASTA reader class
    genomefasta_reader = GenomeFASTA(
        name="seq",
        fasta=input_fasta,
        batch_size=batch_size,
    )
    assert isinstance(genomefasta_reader, GenomeFASTA), "GenomeFASTA reader instantiation failed."

    # Read in the data using the reader
    iterator = genomefasta_reader._reader(
        bed=bed,
        f=pysam.FastaFile(input_fasta)
    )
    _read = [seq.decode('utf-8') for seq in iterator]
    assert np.array_equal(_read, true), "GenomeFASTA reader failed to read in the correct values."

    # Test writing to Zarr
    genomefasta_reader._write(
        fasta_out,
        bed=bed,
        fixed_length=fixed_length,
        sequence_dim="_sequence",
        length_dim=length_dim,
        overwrite=True
    )
    zarr.consolidate_metadata(fasta_out)

    # Test round-trip reading
    data = sd.open_zarr(fasta_out)

    # Verify length and sequence dimensions
    if fixed_length:
        assert data["seq"].shape[1] == fixed_length, "Length dimension is incorrect."
    assert data["seq"].shape[0] == len(bed), "Sequence dimension is incorrect."

    # Verify that the data is the same
    if fixed_length:
        seqs = [''.join(row.astype(str)) for row in data["seq"].values]
    else:
        seqs = data["seq"].values.astype(str)
    print(seqs)
    print(true)
    assert np.array_equal(true, seqs), "Sequences do not match."

    # Clean up temporary files
    for f in temp_dir.iterdir():
        if f.is_dir():
            os.system(f'rm -r {f}')
        else:
            f.unlink()

    print("All tests passed successfully!")

In [5]:
# Test the GenomeFASTA class on variable length sequences
test_GenomeFASTA(variable_fasta_in, variable_bed_in, out, batch_size=50, fixed_length=False, length_dim="_length")

AttributeError: 'DataFrame' object has no attribute 'iter_rows'

In [231]:
# Test the GenomeFASTA class on variable length sequences
test_GenomeFASTA(variable_fasta_in, variable_bed_in, out, batch_size=50, fixed_length=20, length_dim="_length")

100%|██████████| 14/14 [00:00<00:00, 50231.19it/s]

['ATCATCACAACTTGGCCTAA', 'ACATAGTAGGTAACACAACC', 'AGCTCCTTAGcCTGTcAgGG', 'TatAGCCtAAGCcTAACTTC', 'TTTTACACAGCCGCAAGTCT', 'CACCCGTTAAATTTGCCGAG', 'TGGCAACCCGTGTCATGAGG', 'TTATGTAGGCGGGATTCTGA', 'TAAGCGGCCGGTTATGACTA', 'TAGTGAGTAAGAACGATCGC', 'TTTCGACTCGAGATTTATGG', 'CCGAGCTCAGAATACTAAAA', 'TAAGCACAGCATTCTACACA', 'GAGGTAAACCTGACATAGTA']
['ATCATCACAACTTGGCCTAA', 'ACATAGTAGGTAACACAACC', 'AGCTCCTTAGcCTGTcAgGG', 'TatAGCCtAAGCcTAACTTC', 'TTTTACACAGCCGCAAGTCT', 'CACCCGTTAAATTTGCCGAG', 'TGGCAACCCGTGTCATGAGG', 'TTATGTAGGCGGGATTCTGA', 'TAAGCGGCCGGTTATGACTA', 'TAGTGAGTAAGAACGATCGC', 'TTTCGACTCGAGATTTATGG', 'CCGAGCTCAGAATACTAAAA', 'TAAGCACAGCATTCTACACA', 'GAGGTAAACCTGACATAGTA']
All tests passed successfully!


In [232]:
# Test the GenomeFASTA class on fixed length sequences
test_GenomeFASTA(variable_fasta_in, fixed_bed_in, out, batch_size=50, fixed_length=20, length_dim="_length")

100%|██████████| 14/14 [00:00<00:00, 49594.81it/s]

['CGATCATCACAACTTGGCCT', 'GTTAACATAGTAGGTAACAC', 'AACAGCTCCTTAGcCTGTcA', 'TTatAGCCtAAGCcTAACTT', 'TATTTTACACAGCCGCAAGT', 'TACTCACCCGTTAAATTTGC', 'GTTGGCAACCCGTGTCATGA', 'GTTTATGTAGGCGGGATTCT', 'ATAAGCGGCCGGTTATGACT', 'CATAGTGAGTAAGAACGATC', 'ATGGCTTTCGACTCGAGATT', 'CACCGAGCTCAGAATACTAA', 'GTAAGCACAGCATTCTACAC', 'GAGGTAAACCTGACATAGTA']
['CGATCATCACAACTTGGCCT', 'GTTAACATAGTAGGTAACAC', 'AACAGCTCCTTAGcCTGTcA', 'TTatAGCCtAAGCcTAACTT', 'TATTTTACACAGCCGCAAGT', 'TACTCACCCGTTAAATTTGC', 'GTTGGCAACCCGTGTCATGA', 'GTTTATGTAGGCGGGATTCT', 'ATAAGCGGCCGGTTATGACT', 'CATAGTGAGTAAGAACGATC', 'ATGGCTTTCGACTCGAGATT', 'CACCGAGCTCAGAATACTAA', 'GTAAGCACAGCATTCTACAC', 'GAGGTAAACCTGACATAGTA']
All tests passed successfully!


# `BAM`

In [233]:
# list of bams with simulated{1-5}.bam
bams = [Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / f'simulated{i}.bam' for i in range(1, 6)]
bams

[PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/simulated1.bam'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/simulated2.bam'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/simulated3.bam'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/simulated4.bam'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/simulated5.bam')]

In [234]:
# Read in coverage
path_coverage = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'variable.bedcov.pkl'
coverages = pickle.load(open(path_coverage, 'rb'))

In [235]:
# read in the bed file
bed = pd.read_csv(fixed_bed_in, sep="\t", header=None)
bed

,0,1,2
0,chr1,4,24
1,chr1,47,67
2,chr2,46,66
3,chr2,174,194
4,chr3,18,38
5,chr3,78,98
6,chr4,35,55
7,chr4,87,107
8,chr5,40,60
9,chr5,156,176


In [236]:
# Subset the coverage to the bed file coordinates
new_cov = {}
# for each bam file
for bam in bams:
    bam_name = bam.name
    new_cov[bam_name] = {}
    for i, (region, coverage) in enumerate(coverages[bam_name].items()):
        coverage_interval = region.split(":")[1]
        coverage_start, coverage_end = map(int, coverage_interval.split("-"))
        start_offset = coverage_start - bed[1].values[i]
        end_offset = bed[2].values[i] - coverage_end
        new_region = f"{bed[0].values[i]}:{bed[1].values[i]}-{bed[2].values[i]}"
        print(f"Region: {region} -> {new_region}")
        print(coverage)
        print(f"Offsets: {start_offset}, {end_offset}")
        if end_offset == 0:
            new_cov[bam_name][new_region] = coverage[start_offset:]
        else:
            new_cov[bam_name][new_region] = coverage[start_offset:end_offset]

Region: chr1:4-28 -> chr1:4-24
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 3]
Offsets: 0, -4
Region: chr1:47-76 -> chr1:47-67
[3 3 3 2 2 1 2 1 1 1 1 1 2 2 2 2 1 1 1 1 1 2 1 1 1 1 1 1 1]
Offsets: 0, -9
Region: chr2:46-72 -> chr2:46-66
[2 1 1 2 2 2 2 2 2 2 2 3 3 2 4 4 3 3 5 5 6 6 6 6 4 4]
Offsets: 0, -6
Region: chr2:174-197 -> chr2:174-194
[2 1 1 1 2 3 3 4 4 4 5 6 6 6 5 4 3 2 3 4 3 2 3]
Offsets: 0, -3
Region: chr3:18-43 -> chr3:18-38
[1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
Offsets: 0, -5
Region: chr3:78-106 -> chr3:78-98
[1 0 0 1 3 4 5 5 5 5 5 5 5 4 2 1 0 0 1 3 3 3 3 3 3 3 3 3]
Offsets: 0, -8
Region: chr4:35-60 -> chr4:35-55
[1 0 0 0 2 2 3 3 4 4 4 4 4 4 2 2 2 2 1 4 4 4 4 5 5]
Offsets: 0, -5
Region: chr4:87-111 -> chr4:87-107
[2 3 4 4 4 4 3 2 3 3 3 4 4 4 4 4 5 7 6 6 6 4 5 6]
Offsets: 0, -4
Region: chr5:40-62 -> chr5:40-60
[1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 3 3 2 1 1 1 2]
Offsets: 0, -2
Region: chr5:156-181 -> chr5:156-176
[1 2 2 2 2 2 2 2 2 2 3 2 2 4 4 4 6 6 6 6 6 6 6 4 4]
Offs

In [237]:
def test_BAM(bam_files, bed_file, coverages, temp_dir, batch_size=50, sequence_dim="_sequence"):
    """
    Tests the BAM class for one or more input BAM files and their coverage.

    Parameters:
        bam_files (list): List of paths to BAM files.
        bed_file (str or Path): Path to the BED file.
        coverage_file (str or Path): Path to the pickled coverage file.
        temp_dir (str or Path): Directory where temporary files will be stored.
        batch_size (int): Batch size for the BAM reader.
        sequence_dim (str): Name of the sequence dimension in the Zarr output.

    Raises:
        AssertionError: If any test fails.
    """
    temp_dir = Path(temp_dir)
    temp_dir.mkdir(parents=True, exist_ok=True)
    bam_out = temp_dir / 'output.bam.zarr'

    # Test instantiation of the BAM reader class
    bam_reader = BAM(
        name="cov",
        bams=bam_files,
        samples=[bam.name for bam in bam_files],
        batch_size=batch_size,
    )
    assert isinstance(bam_reader, BAM), "BAM reader instantiation failed."

    # Read in the BED file
    bed = pd.read_csv(bed_file, sep="\t", header=None)

    # Verify each BAM file individually
    for bam_file in bam_files:
        print(f"Testing {bam_file}")
        bam_name = bam_file.name
        iterator = bam_reader._reader(bed=bed, f=pysam.AlignmentFile(bam_file))
        _read = [list(seq) for seq in iterator]

        # Verify that the data matches the expected coverage
        for i, (region, coverage) in enumerate(coverages[bam_name].items()):
            print(f"Region {region}: {list(_read[i])}, {coverage}")
            assert np.array_equal(coverage, _read[i]), f"Region {region} in {bam_name} does not match."

    # Test writing to Zarr
    bed["strand"] = "+"
    bam_reader._write(
        bam_out,
        bed=bed,
        fixed_length=False,
        sequence_dim=sequence_dim,
        overwrite=True
    )
    zarr.consolidate_metadata(bam_out)

    # Test round-trip reading
    data = sd.open_zarr(bam_out)
    return data
    # Verify that the data matches the expected coverage for all regions
    for file_index, bam_file in enumerate(bam_files):
        print(f"Testing {bam_file}")
        bam_name = bam_file.name
        for region_index, (region, coverage) in enumerate(coverages[bam_name].items()):
            print(data['cov'][region_index].shape)
            #print(f"Region {region}: {list(data['cov'].values[:, i])}, {coverage}")
            assert np.array_equal(coverage, data["cov"].values[region_index, file_index]), f"Region {region} in {bam_name} does not match."

    # Clean up temporary files
    for f in temp_dir.iterdir():
        if f.is_dir():
            os.system(f'rm -r {f}')
        else:
            f.unlink()

    print("All tests passed successfully!")


In [238]:
data = test_BAM(bams, variable_bed_in, coverages, out, batch_size=50, sequence_dim="_sequence")

Testing /cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/simulated1.bam


100%|██████████| 14/14 [00:00<00:00, 12576.62it/s]


Region chr1:4-28: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3], [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 3]
Region chr1:47-76: [3, 3, 3, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1], [3 3 3 2 2 1 2 1 1 1 1 1 2 2 2 2 1 1 1 1 1 2 1 1 1 1 1 1 1]
Region chr2:46-72: [2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 4, 4, 3, 3, 5, 5, 6, 6, 6, 6, 4, 4], [2 1 1 2 2 2 2 2 2 2 2 3 3 2 4 4 3 3 5 5 6 6 6 6 4 4]
Region chr2:174-197: [2, 1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 5, 4, 3, 2, 3, 4, 3, 2, 3], [2 1 1 1 2 3 3 4 4 4 5 6 6 6 5 4 3 2 3 4 3 2 3]
Region chr3:18-43: [1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
Region chr3:78-106: [1, 0, 0, 1, 3, 4, 5, 5, 5, 5, 5, 5, 5, 4, 2, 1, 0, 0, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3], [1 0 0 1 3 4 5 5 5 5 5 5 5 4 2 1 0 0 1 3 3 3 3 3 3 3 3 3]
Region chr4:35-60: [1, 0, 0, 0, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 1, 4, 4, 4,

100%|██████████| 14/14 [00:00<00:00, 12175.05it/s]


Region chr1:4-28: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0]
Region chr1:47-76: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1], [1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 2 2 2 2 2 2 2 2 1 1 0 0 0 1]
Region chr2:46-72: [2, 2, 1, 2, 2, 2, 3, 4, 4, 4, 3, 3, 3, 2, 2, 3, 2, 2, 3, 3, 3, 4, 4, 4, 4, 3], [2 2 1 2 2 2 3 4 4 4 3 3 3 2 2 3 2 2 3 3 3 4 4 4 4 3]
Region chr2:174-197: [2, 1, 0, 0, 0, 0, 1, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 1, 2, 2, 2, 2, 5], [2 1 0 0 0 0 1 3 3 3 3 3 3 4 4 4 3 1 2 2 2 2 5]
Region chr3:18-43: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 4, 4, 5, 4, 4, 4, 5, 4], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 3 3 4 4 5 4 4 4 5 4]
Region chr3:78-106: [2, 4, 4, 4, 5, 6, 7, 7, 6, 6, 6, 3, 3, 4, 3, 2, 4, 4, 4, 5, 6, 6, 6, 5, 5, 5, 2, 2], [2 4 4 4 5 6 7 7 6 6 6 3 3 4 3 2 4 4 4 5 6 6 6 5 5 5 2 2]
Region chr4:35-60: [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 3, 3, 3, 5, 5, 6, 6, 6, 6,

...

100%|██████████| 14/14 [00:00<00:00, 13446.36it/s]


Region chr1:4-28: [1, 1, 1, 1, 1, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 1, 2, 3, 3, 3, 4], [1 1 1 1 1 2 2 2 3 2 2 2 2 2 3 2 2 2 1 2 3 3 3 4]
Region chr1:47-76: [6, 4, 4, 3, 2, 1, 2, 2, 4, 4, 5, 5, 5, 5, 6, 7, 6, 6, 4, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2], [6 4 4 3 2 1 2 2 4 4 5 5 5 5 6 7 6 6 4 3 2 2 2 3 2 2 2 2 2]
Region chr2:46-72: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 3, 3], [0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 1 1 1 1 1 2 2 3 3]
Region chr2:174-197: [1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2], [1 1 1 2 2 2 2 2 2 2 2 3 3 2 2 2 1 1 2 2 2 2 2]
Region chr3:18-43: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1]
Region chr3:78-106: [1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0], [1 1 2 2 2 2 2 2 2 2 1 1 0 0 0 1 1 2 2 2 2 2 2 2 2 1 1 0]
Region chr4:35-60: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1,

100%|██████████| 14/14 [00:00<00:00, 7639.90it/s]


Region chr1:4-28: [1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 3, 4, 4, 3, 3, 4, 3, 4, 7, 7, 7, 8, 8], [1 1 1 1 2 2 1 2 2 2 2 3 4 4 3 3 4 3 4 7 7 7 8 8]
Region chr1:47-76: [7, 5, 5, 3, 2, 4, 4, 3, 4, 4, 3, 3, 4, 5, 6, 5, 5, 5, 4, 4, 7, 7, 6, 5, 4, 3, 3, 3, 4], [7 5 5 3 2 4 4 3 4 4 3 3 4 5 6 5 5 5 4 4 7 7 6 5 4 3 3 3 4]
Region chr2:46-72: [1, 2, 2, 3, 3, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 4, 5, 5, 6, 5, 6, 6, 5, 5], [1 2 2 3 3 5 4 5 5 5 5 5 5 4 5 3 5 4 5 5 6 5 6 6 5 5]
Region chr2:174-197: [2, 2, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2], [2 2 2 2 1 1 1 1 0 1 1 1 1 2 2 2 2 2 3 2 2 2 2]
Region chr3:18-43: [1, 1, 1, 1, 1, 3, 3, 3, 2, 2, 2, 2, 2, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1 1 1 1 1 3 3 3 2 2 2 2 2 3 3 1 1 1 1 1 1 1 1 0 0]
Region chr3:78-106: [2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 0, 0, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1], [2 2 2 3 3 3 3 3 3 3 1 1 1 0 0 2 2 2 3 3 3 3 3 3 3 1 1 1]
Region chr4:35-60: [0, 1, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 1, 0, 1, 1, 2, 4, 5, 5, 5, 5,

100%|██████████| 14/14 [00:00<00:00, 13394.22it/s]


In [239]:
len(list(coverages["simulated1.bam"].values())[0])

24

In [240]:
len(data.sel(cov_sample="simulated1.bam").cov.values[0])

24

In [241]:
test_BAM([bams[0]], fixed_bed_in, {"simulated1.bam": new_cov["simulated1.bam"]}, out, batch_size=50, sequence_dim="_sequence")

Testing /cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/simulated1.bam


100%|██████████| 14/14 [00:00<00:00, 15208.56it/s]


Region chr1:4-24: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1]
Region chr1:47-67: [3, 3, 3, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1], [3 3 3 2 2 1 2 1 1 1 1 1 2 2 2 2 1 1 1 1]
Region chr2:46-66: [2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 4, 4, 3, 3, 5, 5], [2 1 1 2 2 2 2 2 2 2 2 3 3 2 4 4 3 3 5 5]
Region chr2:174-194: [2, 1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 5, 4, 3, 2, 3, 4], [2 1 1 1 2 3 3 4 4 4 5 6 6 6 5 4 3 2 3 4]
Region chr3:18-38: [1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
Region chr3:78-98: [1, 0, 0, 1, 3, 4, 5, 5, 5, 5, 5, 5, 5, 4, 2, 1, 0, 0, 1, 3], [1 0 0 1 3 4 5 5 5 5 5 5 5 4 2 1 0 0 1 3]
Region chr4:35-55: [1, 0, 0, 0, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 1, 4], [1 0 0 0 2 2 3 3 4 4 4 4 4 4 2 2 2 2 1 4]
Region chr4:87-107: [2, 3, 4, 4, 4, 4, 3, 2, 3, 3, 3, 4, 4, 4, 4, 4, 5, 7, 6, 6], [2 3 4 4 4 4 3 2 3 3 3 4 4 4 4 4 5 7 6 6]
Region chr5:40

100%|██████████| 14/14 [00:00<00:00, 12804.24it/s]


<xarray.Dataset> Size: 128B
Dimensions:          (_sequence: 14, cov_sample: 1)
Coordinates:
  * cov_sample       (cov_sample) object 8B 'simulated1.bam'
Dimensions without coordinates: _sequence
Data variables:
    cov              (_sequence, cov_sample) object 112B dask.array<chunksize=(14, 1), meta=np.ndarray>
    total_reads_cov  (cov_sample) uint64 8B dask.array<chunksize=(1,), meta=np.ndarray>

# `BigWig`

In [275]:
def test_BigWig(bigwig_files, bed_file, chromsizes_file, coverages, temp_dir, batch_size=50, sequence_dim="_sequence", fixed_length=False, length_dim=None):
    """
    Tests the BigWig class for one or more input BigWig files.

    Parameters:
        bigwig_files (list): List of paths to BigWig files.
        bed_file (str or Path): Path to the BED file.
        chromsizes_file (str or Path): Path to the chromsizes file.
        coverage_file (str or Path): Path to the pickled coverage file.
        temp_dir (str or Path): Directory where temporary files will be stored.
        batch_size (int): Batch size for the BigWig reader.
        sequence_dim (str): Name of the sequence dimension in the Zarr output.

    Raises:
        AssertionError: If any test fails.
    """
    temp_dir = Path(temp_dir)
    temp_dir.mkdir(parents=True, exist_ok=True)
    bigwig_out = temp_dir / 'output.bigwig.zarr'

    # Load chromsizes
    chromsizes = {}
    with open(chromsizes_file) as f:
        for line in f:
            chrom, size = line.strip().split()
            chromsizes[chrom] = int(size)

    # Test instantiation of the BigWig reader class
    bigwig_reader = BigWig(
        name="cov",
        bigwigs=bigwig_files,
        samples=[bigwig.name for bigwig in bigwig_files],
        batch_size=batch_size,
    )
    assert isinstance(bigwig_reader, BigWig), "BigWig reader instantiation failed."

    # Read in the BED file
    bed = pd.read_csv(bed_file, sep="\t", header=None)

    # Verify each BigWig file individually
    for bigwig_file in bigwig_files:
        bigwig_name = bigwig_file.name
        iterator = bigwig_reader._reader(
            bed=bed,
            f=pyBigWig.open(str(bigwig_file)),
            contig_lengths=chromsizes
        )
        _read = [list(seq) for seq in iterator]

        # Verify that the data matches the expected coverage
        for i, (region, coverage) in enumerate(coverages[bigwig_name].items()):
            print(f"Region {region}: {list(_read[i])}, {coverage}")
            assert np.array_equal(coverage, _read[i]), f"Region {region} in {bigwig_name} does not match."

        # Close the BigWig file
        iterator.close()

    # Test writing to Zarr
    bed["strand"] = "+"
    bigwig_reader._write(
        bigwig_out,
        bed=bed,
        fixed_length=fixed_length,
        sequence_dim=sequence_dim,
        length_dim=length_dim,
        overwrite=True
    )
    zarr.consolidate_metadata(bigwig_out)

    # Test round-trip reading
    data = sd.open_zarr(bigwig_out)
    return data
    
    # Verify that the data matches the expected coverage for all regions
    for file_index, bigwig_file in enumerate(bigwig_files):
        print(f"Testing {bigwig_file}")
        bigwig_name = bigwig_file.name
        for region_index, (region, coverage) in enumerate(coverages[bigwig_name].items()):
            print(data['cov'][region_index].shape)
            assert np.array_equal(coverage, data["cov"].values[region_index, file_index]), f"Region {region} in {bigwig_name} does not match."

    # Clean up temporary files
    for f in temp_dir.iterdir():
        if f.is_dir():
            os.system(f'rm -r {f}')
        else:
            f.unlink()

    print("All tests passed successfully!")


In [276]:
# list of bigwigs with simulated{1-5}.bw
bigwigs = [Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / f'simulated{i}.bw' for i in range(1, 6)]
chromsizes = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'variable.chrom.sizes'

In [277]:
# modify new cov to have .bw instead of .bam in keys
new_cov_bw = {}
for key, value in new_cov.items():
    new_cov_bw[key.replace(".bam", ".bw")] = value

In [278]:
coverages_bw = {}
for key, value in coverages.items():
    coverages_bw[key.replace(".bam", ".bw")] = value

In [280]:
# Test the BigWig class on variable length sequences
data = test_BigWig(bigwigs, variable_bed_in, chromsizes, coverages_bw, out, batch_size=50, fixed_length=False, sequence_dim="_sequence", length_dim=None)


100%|██████████| 14/14 [00:00<00:00, 19560.38it/s]


Region chr1:4-28: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 3.0], [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 3]
Region chr1:47-76: [3.0, 3.0, 3.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [3 3 3 2 2 1 2 1 1 1 1 1 2 2 2 2 1 1 1 1 1 2 1 1 1 1 1 1 1]
Region chr2:46-72: [2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 2.0, 4.0, 4.0, 3.0, 3.0, 5.0, 5.0, 6.0, 6.0, 6.0, 6.0, 4.0, 4.0], [2 1 1 2 2 2 2 2 2 2 2 3 3 2 4 4 3 3 5 5 6 6 6 6 4 4]
Region chr2:174-197: [2.0, 1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 4.0, 4.0, 4.0, 5.0, 6.0, 6.0, 6.0, 5.0, 4.0, 3.0, 2.0, 3.0, 4.0, 3.0, 2.0, 3.0], [2 1 1 1 2 3 3 4 4 4 5 6 6 6 5 4 3 2 3 4 3 2 3]
Region chr3:18-43: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 


100%|██████████| 14/14 [00:00<00:00, 37568.94it/s]


Region chr1:4-28: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], [1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0]
Region chr1:47-76: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0], [1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 2 2 2 2 2 2 2 2 1 1 0 0 0 1]
Region chr2:46-72: [2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 3.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 2.0, 2.0, 3.0, 2.0, 2.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0], [2 2 1 2 2 2 3 4 4 4 3 3 3 2 2 3 2 2 3 3 3 4 4 4 4 3]
Region chr2:174-197: [2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 1.0, 2.0, 2.0, 2.0, 2.0, 5.0], [2 1 0 0 0 0 1 3 3 3 3 3 3 4 4 4 3 1 2 2 2 2 5]
Region chr3:18-43: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 4.0, 4.0, 5.0, 4.0, 4.0, 4.0, 5.0, 4.0], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 3 3 4 4 5 4 4 4 


100%|██████████| 14/14 [00:00<00:00, 38631.75it/s]


...


100%|██████████| 14/14 [00:00<00:00, 39146.84it/s]


Region chr1:4-28: [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 2.0, 2.0, 1.0, 2.0, 3.0, 3.0, 3.0, 4.0], [1 1 1 1 1 2 2 2 3 2 2 2 2 2 3 2 2 2 1 2 3 3 3 4]
Region chr1:47-76: [6.0, 4.0, 4.0, 3.0, 2.0, 1.0, 2.0, 2.0, 4.0, 4.0, 5.0, 5.0, 5.0, 5.0, 6.0, 7.0, 6.0, 6.0, 4.0, 3.0, 2.0, 2.0, 2.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0], [6 4 4 3 2 1 2 2 4 4 5 5 5 5 6 7 6 6 4 3 2 2 2 3 2 2 2 2 2]
Region chr2:46-72: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 3.0, 3.0], [0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 1 1 1 1 1 2 2 3 3]
Region chr2:174-197: [1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0], [1 1 1 2 2 2 2 2 2 2 2 3 3 2 2 2 1 1 2 2 2 2 2]
Region chr3:18-43: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0], [0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 


100%|██████████| 14/14 [00:00<00:00, 30631.33it/s]


Region chr1:4-28: [1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 3.0, 4.0, 7.0, 7.0, 7.0, 8.0, 8.0], [1 1 1 1 2 2 1 2 2 2 2 3 4 4 3 3 4 3 4 7 7 7 8 8]
Region chr1:47-76: [7.0, 5.0, 5.0, 3.0, 2.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 5.0, 6.0, 5.0, 5.0, 5.0, 4.0, 4.0, 7.0, 7.0, 6.0, 5.0, 4.0, 3.0, 3.0, 3.0, 4.0], [7 5 5 3 2 4 4 3 4 4 3 3 4 5 6 5 5 5 4 4 7 7 6 5 4 3 3 3 4]
Region chr2:46-72: [1.0, 2.0, 2.0, 3.0, 3.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 3.0, 5.0, 4.0, 5.0, 5.0, 6.0, 5.0, 6.0, 6.0, 5.0, 5.0], [1 2 2 3 3 5 4 5 5 5 5 5 5 4 5 3 5 4 5 5 6 5 6 6 5 5]
Region chr2:174-197: [2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 2.0, 2.0, 2.0], [2 2 2 2 1 1 1 1 0 1 1 1 1 2 2 2 2 2 3 2 2 2 2]
Region chr3:18-43: [1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0], [1 1 1 1 1 3 3 3 2 2 2 2 2 3 3 1 1 1 1 1 1 1 1 


100%|██████████| 14/14 [00:00<00:00, 29716.73it/s]

100%|██████████| 14/14 [00:00<00:00, 37617.08it/s]

100%|██████████| 14/14 [00:00<00:00, 38404.35it/s]

100%|██████████| 14/14 [00:00<00:00, 40302.17it/s]

100%|██████████| 14/14 [00:00<00:00, 31621.03it/s]


In [281]:
data

<xarray.Dataset> Size: 600B
Dimensions:     (_sequence: 14, cov_sample: 5)
Coordinates:
  * cov_sample  (cov_sample) object 40B 'simulated1.bw' ... 'simulated5.bw'
Dimensions without coordinates: _sequence
Data variables:
    cov         (_sequence, cov_sample) object 560B dask.array<chunksize=(14, 1), meta=np.ndarray>

In [282]:
cov = data.sel(cov_sample="simulated1.bw").cov.values
cov

ValueError: corrupt buffer, data seem truncated

In [246]:


# Make chromsizes a list of tuples
chromsizes = {}
with open(Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'variable.chrom.sizes') as f:
    for line in f:
        chrom, size = line.strip().split()
        chromsizes[chrom] = int(size)
chromsizes

bigwig_reader = BigWig(
    name="cov",
    bigwigs=bigwigs,
    samples=[bigwig.name for bigwig in bigwigs],
    batch_size=50
)
assert isinstance(bigwig_reader, BigWig), "bigwig reader instantiation failed."

iterator = bigwig_reader._reader(
    bed=bed,
    f=pyBigWig.open(str(bigwigs[0])),
    contig_lengths=chromsizes
)

_read = [list(seq) for seq in iterator]

# Verify that the data matches the expected coverage
for i, (region, coverage) in enumerate(new_cov[bigwigs[0].name.replace(".bw", ".bam")].items()):
    print(f"Region {region}: {list(_read[i])}, {coverage}")
    assert np.array_equal(coverage, _read[i]), f"Region {region} in {bigwigs[0].name} does not match."

100%|██████████| 14/14 [00:00<00:00, 26630.50it/s]

Region chr1:4-24: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1]
Region chr1:47-67: [3.0, 3.0, 3.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0], [3 3 3 2 2 1 2 1 1 1 1 1 2 2 2 2 1 1 1 1]
Region chr2:46-66: [2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 2.0, 4.0, 4.0, 3.0, 3.0, 5.0, 5.0], [2 1 1 2 2 2 2 2 2 2 2 3 3 2 4 4 3 3 5 5]
Region chr2:174-194: [2.0, 1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 4.0, 4.0, 4.0, 5.0, 6.0, 6.0, 6.0, 5.0, 4.0, 3.0, 2.0, 3.0, 4.0], [2 1 1 1 2 3 3 4 4 4 5 6 6 6 5 4 3 2 3 4]
Region chr3:18-38: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
Region chr3:78-98: [1.0, 0.0, 0.0, 1.0, 3.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 2.0, 1.0, 0.0, 0.0, 1.0, 3.0], [1 0 0 1 3 4 5 5 5 5 5 5 5 4 2 1 0 0 1 3]
Region chr4:35-55: [1

# `from_region_files`

In [70]:
from seqdata import from_region_files

In [95]:
def test_from_region_files(
    *readers, 
    names,
    true_seqs,
    true_covs,
    temp_dir, 
    fixed_length=False, 
    length_dim=None
):
    """
    Tests the FlatFASTA class for a single input FASTA file.

    Parameters:
        input_fasta (str or Path): Path to the input FASTA file.
        temp_dir (str or Path): Directory where temporary files will be stored.
        batch_size (int): Batch size for the FlatFASTA reader.
        fixed_length (bool): Whether to use fixed-length sequences when writing to Zarr.
        length_dim (str): Name of the length dimension in the Zarr output.

    Raises:
        AssertionError: If any test fails.
    """
    temp_dir = Path(temp_dir)
    temp_dir.mkdir(parents=True, exist_ok=True)
    out = temp_dir / 'output.zarr'

    data = from_region_files(
        *readers,
        path=out,
        sequence_dim="_sequence",
        fixed_length=fixed_length,
        overwrite=True,
        length_dim=length_dim
    )

    # Verify that the data is the same
    for i, name in enumerate(names):
        if fixed_length:
            seqs = [''.join(row.astype(str)) for row in data[name].values]
        else:
            seqs = data[name].values.astype(str)
        print(name)
        print(seqs)
        print(true[i])
        assert np.array_equal(true[i], seqs), "Sequences do not match."

    # Clean up temporary files
    for f in temp_dir.iterdir():
        print(f)
        if f.is_dir():
            os.system(f'rm -r {f}')
        else:
            f.unlink()

    print("All tests passed successfully!")

In [137]:
fixed_length = 20

outfile = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tmp' / 'output.zarr'

bed = pd.read_csv(fixed_bed_in, sep="\t", header=None)
bed

if fixed_length:
    midpoint = (bed[1] + bed[2]) // 2
    bed[1] = midpoint - fixed_length // 2
    bed[2] = midpoint + fixed_length // 2
bed

fasta_sequences = read_fasta(variable_fasta_in)

true_seqs = [fasta_sequences[chrom][start:end] for chrom, start, end in bed.values]
true_seqs

genome_fasta = GenomeFASTA(
    name="seq",
    fasta=variable_fasta_in,
    batch_size=50,
)
bam = BAM(
    name="cov",
    bams=bams,
    samples=[bam.name for bam in bams],
    batch_size=50,
)

data = from_region_files(
    genome_fasta, 
    bam, 
    bed=fixed_bed_in,
    path=outfile,
    fixed_length=20,
    length_dim="_length",
    sequence_dim="_sequence",
    overwrite=True
)

# Verify length and sequence dimensions
if fixed_length:
    assert data["seq"].shape[1] == fixed_length, "Length dimension is incorrect."
assert data["seq"].shape[0] == len(bed), "Sequence dimension is incorrect."

# Verify that the data is the same
if fixed_length:
    seqs = [''.join(row.astype(str)) for row in data["seq"].values]
else:
    seqs = data["seq"].values.astype(str)
print(seqs)
print(true_seqs)
assert np.array_equal(true_seqs, seqs), "Sequences do not match."

# Verify that the data matches the expected coverage for all regions
for file_index, bam_file in enumerate(bams):
    print(f"Testing {bam_file}")
    bam_name = bam_file.name
    for region_index, (region, coverage) in enumerate(new_cov[bam_name].items()):
        print(data['cov'][region_index].shape)
        print(f"Region {region}: {list(data['cov'].values[region_index, file_index])}, {coverage}")
        assert np.array_equal(coverage, data["cov"].values[region_index, file_index]), f"Region {region} in {bam_name} does not match."

# Clean up temporary files
for f in temp_dir.iterdir():
    if f.is_dir():
        os.system(f'rm -r {f}')
    else:
        f.unlink()

## `read_genome_fasta`

In [151]:
from seqdata import read_genome_fasta

In [160]:
temp_dir = Path(out)
temp_dir.mkdir(parents=True, exist_ok=True)
out = temp_dir / 'output.zarr'

sdata = read_genome_fasta(
    fasta=variable_fasta_in,
    bed=fixed_bed_in,
    out=out,
    name="seq",
    fixed_length=20,
    batch_size=50,
    overwrite=True,
)

# Verify length and sequence dimensions
fasta_sequences = read_fasta(variable_fasta_in)
true = [fasta_sequences[chrom][start:end] for chrom, start, end in bed.values]

if fixed_length:
    assert data["seq"].shape[1] == fixed_length, "Length dimension is incorrect."
assert data["seq"].shape[0] == len(bed), "Sequence dimension is incorrect."

# Verify that the data is the same
if fixed_length:
    seqs = [''.join(row.astype(str)) for row in data["seq"].values]
else:
    seqs = data["seq"].values.astype(str)
assert np.array_equal(true, seqs), "Sequences do not match."

# Clean up temporary files
for f in temp_dir.iterdir():
    if f.is_dir():
        os.system(f'rm -r {f}')
    else:
        f.unlink()

100%|██████████| 14/14 [00:00<00:00, 82472.27it/s]

# `read_bam`

In [150]:
from seqdata import read_bam

In [171]:
temp_dir = Path(out)
temp_dir.mkdir(parents=True, exist_ok=True)
out = temp_dir / 'output.zarr'

sdata = read_bam(
    fasta=variable_fasta_in,
    bed=fixed_bed_in,
    bams=bams,
    out=out,
    seq_name="seq",
    cov_name="cov",
    samples=[b.name for b in bams],
    fixed_length=20,
    batch_size=50,
    overwrite=True,
)

# Verify length and sequence dimensions
fasta_sequences = read_fasta(variable_fasta_in)
true = [fasta_sequences[chrom][start:end] for chrom, start, end in bed.values]

if fixed_length:
    assert data["seq"].shape[1] == fixed_length, "Length dimension is incorrect."
assert data["seq"].shape[0] == len(bed), "Sequence dimension is incorrect."

# Verify that the data is the same
if fixed_length:
    seqs = [''.join(row.astype(str)) for row in data["seq"].values]
else:
    seqs = data["seq"].values.astype(str)
assert np.array_equal(true, seqs), "Sequences do not match."

# Clean up temporary files
for f in temp_dir.iterdir():
    if f.is_dir():
        os.system(f'rm -r {f}')
    else:
        f.unlink()

100%|██████████| 14/14 [00:00<00:00, 11027.28it/s]


100%|██████████| 14/14 [00:00<00:00, 12959.67it/s]


# `read_bigwig`

In [172]:
from seqdata import read_bigwig

In [173]:
[b.name for b in bigwigs]

['simulated1.bw',
 'simulated2.bw',
 'simulated3.bw',
 'simulated4.bw',
 'simulated5.bw']

In [174]:
temp_dir = Path(out)
temp_dir.mkdir(parents=True, exist_ok=True)
out = temp_dir / 'output.zarr'

sdata = read_bigwig(
    fasta=variable_fasta_in,
    bed=fixed_bed_in,
    bigwigs=bigwigs,
    out=out,
    seq_name="seq",
    cov_name="cov",
    samples=[b.name for b in bigwigs],
    fixed_length=20,
    batch_size=50,
    overwrite=True,
)

# Verify length and sequence dimensions
fasta_sequences = read_fasta(variable_fasta_in)
true = [fasta_sequences[chrom][start:end] for chrom, start, end in bed.values]

if fixed_length:
    assert data["seq"].shape[1] == fixed_length, "Length dimension is incorrect."
assert data["seq"].shape[0] == len(bed), "Sequence dimension is incorrect."

# Verify that the data is the same
if fixed_length:
    seqs = [''.join(row.astype(str)) for row in data["seq"].values]
else:
    seqs = data["seq"].values.astype(str)
assert np.array_equal(true, seqs), "Sequences do not match."

# Clean up temporary files
for f in temp_dir.iterdir():
    if f.is_dir():
        os.system(f'rm -r {f}')
    else:
        f.unlink()

100%|██████████| 14/14 [00:00<00:00, 38990.87it/s]


100%|██████████| 14/14 [00:00<00:00, 26985.41it/s]


# DONE!

---

# Testing BAM

In [ ]:
seq_num = 0
print(f"chrom: {bed.iloc[seq_num]['chrom']}, start: {bed.iloc[seq_num]['start']}, end: {bed.iloc[seq_num]['end']}")
cov = bam_reader._count_depth_only(
    f=pysam.AlignmentFile(inbam),
    contig=bed.iloc[seq_num]['chrom'],
    start=bed.iloc[seq_num]['start'],
    end=bed.iloc[seq_num]['end']
)
print(len(cov))
cov

chrom: chr1, start: 10, end: 30
20


array([2, 2, 2, 2, 3, 4, 4, 4, 4, 3, 3, 3, 4, 6, 6, 6, 6, 6, 6, 7],
      dtype=uint16)

In [ ]:
_read = [seq for seq in iterator]
_read

[]

In [ ]:
#inbam = "/cellar/users/aklie/data/datasets/SeqDatasets/K562_ATAC-seq/data/merged.bam"
#inbed = "/cellar/users/aklie/data/datasets/SeqDatasets/K562_ATAC-seq/data/ENCSR868FGK_K562_ATAC-seq_peaks.bed"

In [ ]:
f = pysam.AlignmentFile(inbam)
bed = pd.read_csv(inbed, sep='\t', header=None)

In [ ]:
f.count_coverage(
    contig=bed.iloc[1][0],
    start=bed.iloc[1][1],
    stop=bed.iloc[1][2],
    #read_callback='nofilter',
)

(array('L', [0, 3, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 6, 0, 0, 6, 0, 0, 0]),
 array('L', [0, 0, 2, 0, 2, 2, 0, 0, 4, 5, 0, 0, 0, 0, 6, 0, 0, 0, 6, 6]),
 array('L', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]),
 array('L', [4, 0, 0, 2, 0, 0, 3, 0, 0, 0, 0, 4, 0, 0, 0, 7, 0, 5, 0, 0]))

In [ ]:
# Read in the data using the reader
_read = [seq.decode('utf-8') for seq in iterator]

# Verify that the data is the same
assert np.array_equal(_read, true), "GenomeFASTA reader failed to read in the correct values."

In [ ]:
# Open the BAM file for reading
with pysam.AlignmentFile(inbam, "rb") as bamfile:
    # Iterate over each read
    for read in bamfile.fetch():
        print(f"Read name: {read.query_name}")
        print(f"Reference: {bamfile.get_reference_name(read.reference_id)}")
        print(f"Start position: {read.reference_start}")
        print(f"Read sequence: {read.query_sequence}")
        print(f"CIGAR string: {read.cigarstring}")
        print("------")

Read name: read_chr1_9_19
Reference: chr1
Start position: 9
Read sequence: CCGACTAACT
CIGAR string: 10M
------
Read name: read_chr1_10_20
Reference: chr1
Start position: 10
Read sequence: CGACTAACTG
CIGAR string: 10M
------
Read name: read_chr1_14_24
Reference: chr1
Start position: 14
Read sequence: TAACTGACTG
CIGAR string: 10M
------
Read name: read_chr1_15_25
Reference: chr1
Start position: 15
Read sequence: AACTGACTGA
CIGAR string: 10M
------
Read name: read_chr1_20_30
Reference: chr1
Start position: 20
Read sequence: ACTGATGATG
CIGAR string: 10M
------
Read name: read_chr1_22_32
Reference: chr1
Start position: 22
Read sequence: TGATGATGAT
CIGAR string: 10M
------
Read name: read_chr1_23_33
Reference: chr1
Start position: 23
Read sequence: GATGATGATG
CIGAR string: 10M
------
Read name: read_chr1_23_33
Reference: chr1
Start position: 23
Read sequence: GATGATGATG
CIGAR string: 10M
------
Read name: read_chr1_9_19
Reference: chr1
Start position: 24
Read sequence: ATGATGATGC
CIGAR strin